In [2]:
import pandas as pd
from thefuzz import fuzz
from thefuzz import process

In [8]:
df1 = pd.read_csv('daily_nav - Copy.csv')
df2 = pd.read_csv('amfi_nav.csv')

In [10]:
df1.head(1)

,scheme_code,scheme_name,nav_date,nav
0,48204,360 ONE Balanced Hybrid Fund(G)-Direct Plan,7/22/2024 12:00:00 AM,12.3036


In [4]:
def normalize_name(name):
    return ''.join(e for e in name if e.isalnum()).lower()

In [11]:
df1['normalized_name'] = df1['scheme_name'].apply(normalize_name)
df2['normalized_name'] = df2['scheme_name'].apply(normalize_name)

In [12]:
def match_names(name, choices, scorer=fuzz.token_sort_ratio, cutoff=80):
    match, score = process.extractOne(name, choices, scorer=scorer)
    if score >= cutoff:
        return match
    else:
        return None

In [13]:
normalized_to_original = df2.set_index('normalized_name')['scheme_name'].to_dict()

In [14]:
df1['matched_name'] = df1['normalized_name'].apply(lambda x: match_names(x, normalized_to_original.keys()))

In [17]:
df1.head(1)

,scheme_code,scheme_name,nav_date,nav,normalized_name,matched_name,matched_scheme_name
0,48204,360 ONE Balanced Hybrid Fund(G)-Direct Plan,7/22/2024 12:00:00 AM,12.3036,360onebalancedhybridfundgdirectplan,360onebalancedhybridfunddirectplanidcw,360 ONE Balanced Hybrid Fund - Direct Plan - IDCW


In [20]:
df2.head(1)

,Scheme Code,ISIN Div Payout/ ISIN Growth,ISIN Div Reinvestment,scheme_name,Net Asset Value,Date,normalized_name
0,119551,INF209KA12Z1,INF209KA13Z9,Aditya Birla Sun Life Banking & PSU Debt Fund ...,105.223,22-Jul-24,adityabirlasunlifebankingpsudebtfunddirectidcw


In [24]:
df1['matched_scheme_name'] = df1['matched_name'].map(normalized_to_original)

In [ ]:
# df1['amfi_code'] = df1['matched_name'].map(df2.set_index('normalized_name')['Scheme Code'])

In [25]:
merged_df = df1.merge(df2, left_on='matched_scheme_name', right_on='scheme_name', how='left')

In [ ]:
merged_df = merged_df[['scheme_code', 'scheme_name_x', 'amfi_code']]
merged_df.rename(columns={'scheme_name_x': 'scheme_name'}, inplace=True)

In [28]:
merged_df.head()

,scheme_code,scheme_name_x,nav_date,nav,normalized_name_x,matched_name,matched_scheme_name,Scheme Code,ISIN Div Payout/ ISIN Growth,ISIN Div Reinvestment,scheme_name_y,Net Asset Value,Date,normalized_name_y
0,48204,360 ONE Balanced Hybrid Fund(G)-Direct Plan,7/22/2024 12:00:00 AM,12.3036,360onebalancedhybridfundgdirectplan,360onebalancedhybridfunddirectplanidcw,360 ONE Balanced Hybrid Fund - Direct Plan - IDCW,152073.0,INF579M01AZ6,INF579M01BA7,360 ONE Balanced Hybrid Fund - Direct Plan - IDCW,12.3036,22-Jul-24,360onebalancedhybridfunddirectplanidcw
1,48205,360 ONE Balanced Hybrid Fund(IDCW)-Direct Plan,7/22/2024 12:00:00 AM,12.3036,360onebalancedhybridfundidcwdirectplan,360onebalancedhybridfunddirectplanidcw,360 ONE Balanced Hybrid Fund - Direct Plan - IDCW,152073.0,INF579M01AZ6,INF579M01BA7,360 ONE Balanced Hybrid Fund - Direct Plan - IDCW,12.3036,22-Jul-24,360onebalancedhybridfunddirectplanidcw
2,48144,360 ONE Balanced Hybrid Fund-Reg(G),7/22/2024 12:00:00 AM,12.1531,360onebalancedhybridfundregg,360onebalancedhybridfundregularplangrowth,360 ONE Balanced Hybrid Fund - Regular Plan - ...,152075.0,INF579M01AV5,-,360 ONE Balanced Hybrid Fund - Regular Plan - ...,12.1531,22-Jul-24,360onebalancedhybridfundregularplangrowth
3,48203,360 ONE Balanced Hybrid Fund-Reg(IDCW),7/22/2024 12:00:00 AM,12.1531,360onebalancedhybridfundregidcw,360onebalancedhyrbridfundregularplanidcw,360 ONE Balanced Hyrbrid fund - Regular Plan -...,152074.0,INF579M01AW3,INF579M01AX1,360 ONE Balanced Hyrbrid fund - Regular Plan -...,12.1531,22-Jul-24,360onebalancedhyrbridfundregularplanidcw
4,22085,360 ONE Dynamic Bond Fund(G)-Direct Plan,7/22/2024 12:00:00 AM,21.7659,360onedynamicbondfundgdirectplan,360onedynamicbondfunddirectplangrowth,360 ONE Dynamic Bond Fund Direct Plan Growth,122715.0,INF579M01266,-,360 ONE Dynamic Bond Fund Direct Plan Growth,21.7659,22-Jul-24,360onedynamicbondfunddirectplangrowth


In [29]:
merged_df.to_csv('merged_table.csv', index=False)